In [1]:
!docker run --name redis-test -d -p 6379:6379 redis

Unable to find image 'redis:latest' locally
latest: Pulling from library/redis

ecc8b0af: Pulling fs layer 
2eb8b06c: Pulling fs layer 
8a492665: Pulling fs layer 
7b7cdd6d: Pulling fs layer 
6557134d: Pulling fs layer 
b700ef54: Pulling fs layer 
Digest: sha256:acb90ced0bd769b7c04cb4c32c4494ba7b3e0ee068bdbfff0eeb0d31c2a210787AExtracting  5.308MB/28.1MB
Status: Downloaded newer image for redis:latest
9602244f8503a68d78affe3daf18dde16e300173257ec554986a430bdbee3aa3


In [2]:
!pip install redis

In [ ]:
import redis

r = redis.Redis(host='localhost', port=6379, db=0)
r.set("mykey", "Hello from Python")
value = r.get("mykey")
print(value.decode("utf-8"))

Hello from Python


In [12]:
r.set("mykey", " from Python")
r.get("mykey")

b' from Python'

In [13]:
# 全件取得
keys = r.keys('*')
print("Keys:", keys)
print("Values:", [r.get(key).decode("utf-8") for key in keys])

Keys: [b'mykey']
Values: [' from Python']


In [ ]:
# Increment
r.incr("counter")  # 1
r.incr("counter")  # 2
r.incr("counter")  # 3
print(r.get("counter"))  # b'3'

b'3'


In [18]:
# List型
r.lpush("mylist", "one")
r.lpush("mylist", "two")
r.lpush("mylist", "three")

print(r.lrange("mylist", 0, -1))  

[b'three', b'two', b'one', b'three', b'two', b'one']


In [17]:
# Set型
r.sadd("myset", "apple")
r.sadd("myset", "banana")
r.sadd("myset", "apple")

print(r.smembers("myset"))

{b'apple', b'banana'}


In [32]:
# Hash型
r.hset("user:1000", mapping={"name": "Alice", "age": "30"})
r.hset("user:1000", "email", "aa@example.com")
r.hset("user:1001", mapping={"name": "Bob", "age": "25"})

print(r.hget("user:1000", "name").decode("utf-8"))
print(r.hgetall("user:1000"))
print(r.hgetall("user:1001"))

Alice
{b'name': b'Alice', b'age': b'30', b'email': b'aa@example.com'}
{b'name': b'Bob', b'age': b'25'}


In [27]:
for k in r.keys('*'):
    print(k, r.type(k), r.get(k) if r.type(k) == b'string' else None)

b'counter' b'string' b'3'
b'myset' b'set' None
b'user:1001' b'hash' None
b'mykey' b'string' b' from Python'
b'mylist' b'list' None
b'user:1000' b'hash' None


In [34]:
# スコア付きデータを追加
r.zadd("leaderboard", {"Alice": 100, "Bob": 200, "Charlie": 150})

print(r.zrange("leaderboard", 0, -1, withscores=True))
print(r.zrevrange("leaderboard", 0, -1, withscores=True))

print(r.zscore("leaderboard", "Alice"))  # 100.0
print(r.zrank("leaderboard", "Alice"))   # 0
print(r.zrevrank("leaderboard", "Alice"))  # 2

[(b'Alice', 100.0), (b'Charlie', 150.0), (b'Bob', 200.0)]
[(b'Bob', 200.0), (b'Charlie', 150.0), (b'Alice', 100.0)]
100.0
0
2


# Pub/Sub的使い方

In [ ]:
# pub/sub

def message_handler(message):
    print("Received message:", message['data'])
pubsub = r.pubsub()
pubsub.subscribe(**{'mychannel': message_handler})
pubsub_thread = pubsub.run_in_thread(sleep_time=0.001)

r.publish('mychannel', 'Hello, Redis!')
r.publish('mychannel', 'Another message')

# for message in pubsub.listen():
#     print(message)

Received message: b'Hello, Redis!'
Received message: b'Another message'


1

# Stream（ログ・メッセージキュー）

In [36]:
r.xadd("mystream", {"message": "Hello"})
r.xadd("mystream", {"message": "World"})

print(r.xrange("mystream", min="-", max="+"))

[(b'1758278996196-0', {b'message': b'Hello'}), (b'1758278996200-0', {b'message': b'World'})]
